# LeNet Lab Solution
![LeNet Architecture](lenet.png)
Source: Yan LeCun

## Load Data

Load the MNIST data, which comes pre-loaded with TensorFlow.

You do not need to modify this section.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

#mnist = input_data.read_data_sets("MNIST_data/", reshape=False)
#X_train, y_train           = mnist.train.images, mnist.train.labels
#X_validation, y_validation = mnist.validation.images, mnist.validation.labels
#X_test, y_test             = mnist.test.images, mnist.test.labels
# Load pickled data
import pickle
import cv2
import csv
import time
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.contrib.layers import flatten
# TODO: Fill this in based on where you saved the training and testing data

training_file = r'train.p'
validation_file= r'valid.p'
testing_file = r'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_validation = X_valid
y_validation = y_valid
X_test, y_test = test['features'], test['labels']



X_train = np.sum(X_train/3, axis=3, keepdims=True)
#y_train = np.sum(y_train/3, axis=3, keepdims=True)
X_validation = np.sum(X_validation/3, axis=3, keepdims=True)
#y_validation = np.sum(y_validation/3, axis=3, keepdims=True)
X_test = np.sum(X_test/3, axis=3, keepdims=True)
#y_test = np.sum(y_test/3, axis=3, keepdims=True)
assert(len(X_train) == len(y_train))
assert(len(X_validation) == len(y_validation))
assert(len(X_test) == len(y_test))

print()
print("Image Shape: {}".format(X_train[0].shape))
print()
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))


Image Shape: (32, 32, 1)

Training Set:   34799 samples
Validation Set: 4410 samples
Test Set:       12630 samples


The MNIST data that TensorFlow pre-loads comes as 28x28x1 images.

However, the LeNet architecture only accepts 32x32xC images, where C is the number of color channels.

In order to reformat the MNIST data into a shape that LeNet will accept, we pad the data with two rows of zeros on the top and bottom, and two columns of zeros on the left and right (28+2+2 = 32).

You do not need to modify this section.

In [ ]:
import numpy as np

# Pad images with 0s
#X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
#X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
#X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))

## Visualize Data

View a sample from the dataset.

You do not need to modify this section.

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

index = random.randint(0, len(X_train))
image = X_train[index].squeeze()

plt.figure(figsize=(1,1))
plt.imshow(image, cmap="gray")
print(y_train[index])

## Preprocess Data

Shuffle the training data.

You do not need to modify this section.

In [ ]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

## Setup TensorFlow
The `EPOCH` and `BATCH_SIZE` values affect the training speed and model accuracy.

You do not need to modify this section.

In [9]:
import tensorflow as tf

EPOCHS = 400#60
BATCH_SIZE = 1024

## SOLUTION: Implement LeNet-5
Implement the [LeNet-5](http://yann.lecun.com/exdb/lenet/) neural network architecture.

This is the only cell you need to edit.
### Input
The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

### Architecture
**Layer 1: Convolutional.** The output shape should be 28x28x6.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using `tf.contrib.layers.flatten`, which is already imported for you.

**Layer 3: Fully Connected.** This should have 120 outputs.

**Activation.** Your choice of activation function.

**Layer 4: Fully Connected.** This should have 84 outputs.

**Activation.** Your choice of activation function.

**Layer 5: Fully Connected (Logits).** This should have 10 outputs.

### Output
Return the result of the 2nd fully connected layer.

In [3]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 10), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(10))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

## Features and Labels
Train LeNet to classify [MNIST](http://yann.lecun.com/exdb/mnist/) data.

`x` is a placeholder for a batch of input images.
`y` is a placeholder for a batch of output labels.

You do not need to modify this section.

In [4]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 10)

## Training Pipeline
Create a training pipeline that uses the model to classify MNIST data.

You do not need to modify this section.

In [5]:
rate = 0.001#09#100.1001

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

## Model Evaluation
Evaluate how well the loss and accuracy of the model for a given dataset.

You do not need to modify this section.

In [6]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

## Train the Model
Run the training data through the training pipeline to train the model.

Before each epoch, shuffle the training set.

After each epoch, measure the loss and accuracy of the validation set.

Save the model after training.

You do not need to modify this section.

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Validation Accuracy = 0.071

EPOCH 2 ...
Validation Accuracy = 0.034

EPOCH 3 ...
Validation Accuracy = 0.646

EPOCH 4 ...
Validation Accuracy = 0.034

EPOCH 5 ...
Validation Accuracy = 0.034

EPOCH 6 ...
Validation Accuracy = 0.034

EPOCH 7 ...
Validation Accuracy = 0.034

EPOCH 8 ...
Validation Accuracy = 0.646

EPOCH 9 ...
Validation Accuracy = 0.034

EPOCH 10 ...
Validation Accuracy = 0.048

EPOCH 11 ...
Validation Accuracy = 0.646

EPOCH 12 ...
Validation Accuracy = 0.646

EPOCH 13 ...
Validation Accuracy = 0.034

EPOCH 14 ...
Validation Accuracy = 0.054

EPOCH 15 ...
Validation Accuracy = 0.034

EPOCH 16 ...
Validation Accuracy = 0.034

EPOCH 17 ...
Validation Accuracy = 0.034

EPOCH 18 ...
Validation Accuracy = 0.048

EPOCH 19 ...
Validation Accuracy = 0.054

EPOCH 20 ...
Validation Accuracy = 0.048

EPOCH 21 ...
Validation Accuracy = 0.054

EPOCH 22 ...
Validation Accuracy = 0.034

EPOCH 23 ...
Validation Accuracy = 0.054

EPOCH 24 ...
Validation Accura

EPOCH 194 ...
Validation Accuracy = 0.034

EPOCH 195 ...
Validation Accuracy = 0.034

EPOCH 196 ...
Validation Accuracy = 0.034

EPOCH 197 ...
Validation Accuracy = 0.034

EPOCH 198 ...
Validation Accuracy = 0.034

EPOCH 199 ...
Validation Accuracy = 0.034

EPOCH 200 ...
Validation Accuracy = 0.034

EPOCH 201 ...
Validation Accuracy = 0.054

EPOCH 202 ...
Validation Accuracy = 0.048

EPOCH 203 ...
Validation Accuracy = 0.054

EPOCH 204 ...
Validation Accuracy = 0.014

EPOCH 205 ...
Validation Accuracy = 0.034

EPOCH 206 ...
Validation Accuracy = 0.048

EPOCH 207 ...
Validation Accuracy = 0.054

EPOCH 208 ...
Validation Accuracy = 0.048

EPOCH 209 ...
Validation Accuracy = 0.048

EPOCH 210 ...
Validation Accuracy = 0.014

EPOCH 211 ...
Validation Accuracy = 0.034

EPOCH 212 ...
Validation Accuracy = 0.034

EPOCH 213 ...
Validation Accuracy = 0.048

EPOCH 214 ...
Validation Accuracy = 0.034

EPOCH 215 ...
Validation Accuracy = 0.034

EPOCH 216 ...
Validation Accuracy = 0.034

EPOCH 217 .

EPOCH 385 ...
Validation Accuracy = 0.646

EPOCH 386 ...
Validation Accuracy = 0.034

EPOCH 387 ...
Validation Accuracy = 0.014

EPOCH 388 ...
Validation Accuracy = 0.034

EPOCH 389 ...
Validation Accuracy = 0.054

EPOCH 390 ...
Validation Accuracy = 0.034

EPOCH 391 ...
Validation Accuracy = 0.034

EPOCH 392 ...
Validation Accuracy = 0.048

EPOCH 393 ...
Validation Accuracy = 0.054

EPOCH 394 ...
Validation Accuracy = 0.054

EPOCH 395 ...
Validation Accuracy = 0.048

EPOCH 396 ...
Validation Accuracy = 0.034

EPOCH 397 ...
Validation Accuracy = 0.054

EPOCH 398 ...
Validation Accuracy = 0.034

EPOCH 399 ...
Validation Accuracy = 0.048

EPOCH 400 ...
Validation Accuracy = 0.014

Model saved


## Evaluate the Model
Once you are completely satisfied with your model, evaluate the performance of the model on the test set.

Be sure to only do this once!

If you were to measure the performance of your trained model on the test set, then improve your model, and then measure the performance of your model on the test set again, that would invalidate your test results. You wouldn't get a true measure of how well your model would perform against real data.

You do not need to modify this section.

In [11]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./lenet
Test Accuracy = 0.012
